In [2]:
import os
import sys
import time
import serial
import math

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'joachim'))
from Thymio import Thymio
#ls /dev/ttyACM*
#sudo chmod 666 /dev/ttyACM0

In [3]:
class robot():
    def __init__(self):
        self.pos_th = [0,0,0]
        self.th = Thymio.serial(port="/dev/ttyACM0", refreshing_rate=0.1) #/dev/ttyACM0 for linux
    def GetPosition(self):
        return self.pos_th

In [4]:
def Main():
    
    th_m = robot()
    pos_th = th_m.getPosition()

if __name__=="__main__": 
    Main() 

SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'